# Predicting the Success of a Kickstarter Campaign
Part B. Initial data preparation and loading into postgresql on AWS instance.

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import os
import json

In [2]:
# import sys
# sys.executable

In [3]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

def to_str(obj):
    return str(obj).split(' ')[0]

def count_list_items(list):
    try:
        return len(list)
    except:
        return 0

In [4]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

In [5]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [6]:
# data at WebRobots https://webrobots.io/kickstarter-datasets/
csv_range = range(1, 49)

df = pd.DataFrame()
# files in monthly batches; this loads the files within csv_range
for csv in csv_range:
    csv = str(csv).zfill(3)
    path ='data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter{}.csv'.format(csv)
    frame = pd.read_csv(path) #,index_col=None)

    frame[csv] = csv
    df = df.append(frame)
    df = df.drop([csv],1)
print(df.shape)
df.groupby('state').nunique()

(192716, 37)


/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/pandas/core/groupby.py:3406: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
state,,,,,,,,,,,,,,,,,,,,,
canceled,270,8622,122,2051,22,8693,8566,14,6,2,...,8693,122,1,2,1,8692,1959,8693,3143,1
failed,482,71393,147,7107,22,71719,72168,14,6,2,...,71730,147,1,2,1,70488,6027,71730,20315,2
live,355,2960,166,1569,22,2959,5350,14,6,2,...,2960,165,1,2,1,2959,370,3346,1985,2
successful,3101,91357,169,28609,22,91668,105474,14,6,2,...,91695,168,1,2,1,86230,5548,91718,49498,1
suspended,79,587,99,208,19,592,591,12,5,2,...,592,99,1,1,1,592,191,592,248,1


In [7]:
# remove cancelled, suspended and live
df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]
df['state'] = df['state'].replace({"failed":0,'successful':1})
df['spotlight'] = df['spotlight'].replace({False:0,True:1})

In [8]:
df = df[['backers_count','blurb', 'category', 'converted_pledged_amount', 'country','created_at', 
         'creator', 'currency', 'currency_symbol','currency_trailing_code', 'current_currency', 
         'deadline','disable_communication', 'friends', 'fx_rate', 'goal', 'id','is_backing', 
         'is_starrable', 'is_starred', 'launched_at', 'location','name', 'permissions', 'photo', 
         'pledged', 'profile', 'slug','source_url', 'spotlight','staff_pick', 'state', 'state_changed_at',
        'static_usd_rate', 'urls', 'usd_pledged', 'usd_type']]


In [9]:
# remove duplicate ID entries
df = df.drop_duplicates(subset='id')

In [10]:
df['category_main'] = [json.loads(x)['urls']['web']['discover'][:].split('/')[5] for x in df['category']]
df['category_main'] = df['category_main'].replace({'film%20&%20video': 'film_and_video'})

In [11]:
df['category_name'] = [json.loads(x)['name'] for x in df['category']]
df['creator_name'] = [json.loads(x)['name'] for x in df['creator']]
df['blurb_len2'] = df['blurb'].str.lower().str.split()
df['blurb_length'] = df['blurb_len2'].apply(count_list_items)

In [12]:
df['created_at'] = pd.to_datetime(df['created_at'],unit='s')
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')
df['deadline'] = pd.to_datetime(df['deadline'],unit='s')

df['created'] = pd.to_datetime(df['created_at'])
df['launched'] = pd.to_datetime(df['launched_at'])
df['deadline'] = pd.to_datetime(df['deadline'])

df['created'] = df.created.apply(lambda x: x.date())
df['launched'] = df.launched.apply(lambda x: x.date())
df['deadline'] = df.deadline.apply(lambda x: x.date())

df.staff_pick = df.staff_pick.astype(int)

df['campaign_length'] = (df['deadline'] - df['launched']).apply(to_str)

In [13]:
print(df.shape)

num_cols = ['backers_count',
            'pledged',
            'usd_pledged',
            'goal',
            'campaign_length',
            'blurb_length',
            'fx_rate']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
df['usd_goal'] = df['goal'] * (df['usd_pledged']/df['pledged'])
df['pct_goal_achieved'] = round((df['usd_pledged'] / df['usd_goal']),1)

(163425, 45)


In [14]:
df = df.sort_values(['deadline'], ascending=[False])


In [15]:
df = df[['id','name','state','category_main','category_name','backers_count','pct_goal_achieved','usd_pledged','usd_goal','country','currency','campaign_length',
         'deadline','launched','created','staff_pick','spotlight','creator_name','blurb_length']]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163425 entries, 543 to 980
Data columns (total 19 columns):
id                   163425 non-null int64
name                 163424 non-null object
state                163425 non-null int64
category_main        163425 non-null object
category_name        163425 non-null object
backers_count        163425 non-null int64
pct_goal_achieved    147802 non-null float64
usd_pledged          163425 non-null float64
usd_goal             147802 non-null float64
country              163425 non-null object
currency             163425 non-null object
campaign_length      163425 non-null int64
deadline             163425 non-null object
launched             163425 non-null object
created              163425 non-null object
staff_pick           163425 non-null int64
spotlight            163425 non-null int64
creator_name         163425 non-null object
blurb_length         163425 non-null int64
dtypes: float64(3), int64(7), object(9)
memory usage: 24.9

In [17]:
df.tail()

,id,name,state,category_main,category_name,backers_count,pct_goal_achieved,usd_pledged,usd_goal,country,currency,campaign_length,deadline,launched,created,staff_pick,spotlight,creator_name,blurb_length
1785,1578671837,Support casting my sculpture in bronze,0,art,Sculpture,1,0.0,25.0,2000.0,US,USD,30,2009-05-31,2009-05-01,2009-05-01,0,0,C.K. Sample,18
2352,266044220,Help me write my second novel.,1,publishing,Fiction,18,1.1,563.0,500.0,US,USD,30,2009-05-29,2009-04-29,2009-04-29,1,1,Brendan Mackie,24
2373,199916122,Mr. Squiggles,0,art,Illustration,0,NaN,0.0,NaN,US,USD,10,2009-05-22,2009-05-12,2009-05-12,0,0,C.K. Sample,22
0,2089078683,New York Makes a Book!!,1,journalism,Journalism,110,1.1,3329.0,3000.0,US,USD,18,2009-05-16,2009-04-28,2009-04-27,1,1,We Make a Book,22
980,1703704063,drawing for dollars,1,art,Illustration,3,1.8,35.0,20.0,US,USD,9,2009-05-03,2009-04-24,2009-04-24,1,1,darkpony,26


In [18]:
df.columns

Index(['id', 'name', 'state', 'category_main', 'category_name',
       'backers_count', 'pct_goal_achieved', 'usd_pledged', 'usd_goal',
       'country', 'currency', 'campaign_length', 'deadline', 'launched',
       'created', 'staff_pick', 'spotlight', 'creator_name', 'blurb_length'],
      dtype='object')

In [19]:
df.head()

,id,name,state,category_main,category_name,backers_count,pct_goal_achieved,usd_pledged,usd_goal,country,currency,campaign_length,deadline,launched,created,staff_pick,spotlight,creator_name,blurb_length
543,1396766240,Ripple Playing Cards - Printed by USPCC,0,games,Playing Cards,131,0.3,3387.000000,9999.000000,US,USD,38,2018-01-12,2017-12-05,2017-10-08,0,0,B.Y. Eidelman,16
3545,2065169465,Cotton-Top Pastries,1,food,Small Batch,99,1.3,9858.000000,7500.000000,US,USD,30,2018-01-12,2017-12-13,2017-12-12,1,1,Holly Weist,5
130,1647325451,Code Switch,1,film_and_video,Horror,34,1.5,4611.000000,3000.000000,US,USD,32,2018-01-12,2017-12-11,2017-11-10,0,1,Alba Roland,23
53,727157486,Rain Dog Farm,0,food,Farms,49,0.3,4741.000000,18000.000000,US,USD,38,2018-01-12,2017-12-05,2017-11-28,1,0,Charlie Wainger,9
3146,1756145145,WANGTA: a novel,1,publishing,Fiction,13,1.0,427.185132,427.185132,CA,CAD,21,2018-01-12,2017-12-22,2017-12-18,0,1,D. H. de Bruin,22


In [20]:
print(df.shape)
df.groupby('state').nunique()

(163425, 19)


,id,name,state,category_main,category_name,backers_count,pct_goal_achieved,usd_pledged,usd_goal,country,currency,campaign_length,deadline,launched,created,staff_pick,spotlight,creator_name,blurb_length
state,,,,,,,,,,,,,,,,,,,
0,71730,71593,1,15,137,482,12,20315,15546,22,14,92,2975,3013,3011,2,1,65012,35
1,91695,91531,1,15,159,3101,729,49496,16718,22,14,92,3077,3122,3132,2,1,76695,35


In [21]:
df.to_pickle('data/kickstarter_data_ds2.pkl')
try:
    df.to_sql("kickstarter_data_ds2", engine, chunksize=20000, if_exists='replace')
except Exception as e:
    print(e)
    pass